In [15]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Load the enrolments data to count student enrollments for each exam
enrolments_data = pd.read_csv(
    'nott/enrolements', sep=' ', header=None, names=['student_code', 'exam_code'])

# Count students enrolled in each exam
exam_enrollment_count = enrolments_data['exam_code'].value_counts().to_dict()

# Define the timeslots and their durations
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
times = [("9:00", 180), ("13:30", 120), ("16:30", 120)]
day_ids = [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13]

timeslots = []
timeslot_labels = {}
timeslot_id = 1

for day_id, day in zip(day_ids, days * 2):
    if day == "Saturday":
        timeslots.append((timeslot_id, day_id, f"{day} 9:00", 180))
        timeslot_labels[timeslot_id] = f"{day} 9:00"
        timeslot_id += 1
    else:
        for time, duration in times:
            timeslots.append((timeslot_id, day_id, f"{day} {time}", duration))
            timeslot_labels[timeslot_id] = f"{day} {time}"
            timeslot_id += 1

# Function to load the solution file
def load_solution(file_path):
    df = pd.read_csv(file_path)
    # Add student counts to the DataFrame
    df['student_count'] = df['exam'].apply(lambda x: exam_enrollment_count.get(x.upper(), 0))
    return df

# Function to plot the solution
def plot_solution(df):
    # Extract unique timeslots
    timeslots = df['timeslot'].unique()

    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(15, 8))

    # Plot the stacked bar chart
    for timeslot in timeslots:
        timeslot_data = df[df['timeslot'] == timeslot]
        bottom = 0
        for exam in timeslot_data['exam'].unique():
            exam_data = timeslot_data[timeslot_data['exam'] == exam]
            ax.bar(timeslot, exam_data['student_count'].values[0], bottom=bottom, label=exam)
            bottom += exam_data['student_count'].values[0]

    # Add a dotted line for the capacity
    ax.axhline(y=1550, color='r', linestyle='--', label='Capacity (1550)')

    # Set labels and title
    ax.set_xlabel('Timeslot')
    ax.set_ylabel('Number of Students')
    ax.set_title('Exam Schedule')

    # Set custom x-axis labels
    ax.set_xticks(timeslots)
    ax.set_xticklabels([timeslot_labels[t] for t in timeslots], rotation=45, ha='right')

    # Remove the legend
    ax.legend().remove()

    # Show the plot
    plt.show()

# Function to update the plot based on the selected file
def update_plot(file_path):
    df = load_solution(file_path)
    plot_solution(df)

# Get the list of solution files
solution_files = [os.path.join(root, file)
                  for root, dirs, files in os.walk('results')
                  for file in files if file.endswith('.csv')]

# Create a dropdown widget for file selection
file_selector = widgets.Dropdown(
    options=solution_files,
    description='Solution File:',
    disabled=False,
)

# Use the interact function to update the plot based on the selected file
interact(update_plot, file_path=file_selector)

interactive(children=(Dropdown(description='Solution File:', options=('results\\20240902_153229_tmax23_capacit…

<function __main__.update_plot(file_path)>